In [1]:
import os
import sys
current_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
from pixel_setter import play_video
from scipy.ndimage import uniform_filter
from EMA_functions import *
from Feature_selecter import *
import matplotlib.animation as animation
from pycine.raw import read_frames
from pyMRAW import save_mraw
import glob
root_drive_sim = os.path.normpath('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations')
# root_video = 'E:/FreeFreeBeam4-16-24/cine rgb'
# root_mraw = os.path.normpath('E:/FreeFreeBeam4-16-24/mraw')
# root_video = os.path.normpath('D:/thijsmas/wing')
# root_mraw = os.path.normpath('D:/thijsmas/wing/mraw')

root_video = os.path.normpath('E:/AA598/rgb')
root_mraw = os.path.normpath('E:/AA598/mraw')
path_list = glob.glob(os.path.join(root_video, '*.cine'))

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
def set_up_reader(cine_file, start_frame=1, count=None):
    raw_images, setup, bpp = read_frames(cine_file, start_frame=start_frame, count=count)        
    info = {'Record Rate(fps)': setup.FrameRate,
            # 'Total Frame': setup.ImageCount,
            'Image Width': setup.ImWidth,
            'Image Height': setup.ImHeight,
            'File Format': 'mraw',
            # 'EffectiveBit Depth': bpp,
            'Shutter Speed(s)': setup.Shutter/1e6,
            'Post trigger': setup.PostTrigger}
    return raw_images, info

def get_frames_numpy(raw_images):
    return np.array([raw_image for raw_image in raw_images])

def get_frames_memmap(raw_images, n_frames, filename='memmapped.dat'):
    # Determine the shape and dtype of the array
    sample_image = next(iter(raw_images))
    shape = (n_frames,) + sample_image.shape
    dtype = sample_image.dtype

    # Create a memory-mapped array with the desired shape and dtype
    memmapped_array = np.memmap(filename, dtype=dtype, mode='w+', shape=shape)

    # Fill the memory-mapped array with the images
    print_i = int(n_frames/10)
    for i, raw_image in enumerate(raw_images):
        memmapped_array[i] = raw_image
        if i % print_i == 0:
            print(f'Frame {i+1}/{n_frames} written to memory-mapped array')
        if i == n_frames-1:
            break

    return memmapped_array, i

In [3]:
path_list

['E:\\AA598\\rgb\\impact_test_with_cam_4_24.cine',
 'E:\\AA598\\rgb\\impact_test_with_cam_4_24_zoom.cine',
 'E:\\AA598\\rgb\\impact_test_with_cam_4_24_loc2.cine']

In [5]:
count = None
n_frames_max_vec = [95000, 13000, 33000]
for path_c, n_frames_max in zip(path_list, n_frames_max_vec):
    file_name = os.path.basename(path_c)
    file_path = os.path.join(root_video, file_name)
    base_path = os.path.join(root_mraw, file_name[:-5])
    mraw_path = base_path + '.mraw'
    cih_path = base_path + '.cih'
    mem_path = base_path + '.dat'
    if os.path.exists(cih_path):
        print("The file already exists")
        continue
        
    raw_images, info = set_up_reader(file_path, start_frame=1, count=count)
    # frames, n_frames = get_frames_memmap(raw_images, n_frames_max, filename=mem_path)
    frames = get_frames_numpy(raw_images)
    
    if os.path.exists(mraw_path):
        os.remove(mraw_path)
    if os.path.exists(cih_path):
        os.remove(cih_path)

    # save_mraw(frames[:n_frames], mraw_path, bit_depth=16, ext='mraw', info_dict=info) #, info_dict=info
    save_mraw(frames, mraw_path, bit_depth=16, ext='mraw', info_dict=info) #, info_dict=info
    if os.path.exists(mem_path):
        frames.flush()
        del frames
        os.remove(mem_path)
    print(f'File {file_name} saved')
# video = pyidi.pyIDI(frames)
# del frames
# video.info = info
# video.root = root_video
# EMA_structure = EMA_Structure(file_name)
# EMA_structure.fs_camera = video.info['Record Rate(fps)']
# EMA_structure.t_camera_raw = np.arange(video.info['Total Frame']) / EMA_structure.fs_camera

File impact_test_with_cam_4_24.cine saved
File impact_test_with_cam_4_24_zoom.cine saved
File impact_test_with_cam_4_24_loc2.cine saved


In [ ]:
EMA_structure = EMA_Structure(file_name[:-5] + '.cih')
video = EMA_structure.open_video(add_extension=False)
video

In [ ]:
file_name